In [85]:
%matplotlib widget
import numpy as np
import pandas as pd
import random 
import talib as ta
import math
# from numba import jit

from importlib import reload 
from qflib import basic
reload(basic)

global engine, conn
engine = basic.engine()
conn = basic.conn(engine)

df=pd.DataFrame()  # 初始化空值
vTest = 1000

''' 清空 stock_cal table '''
sql = "truncate table day_zhibiao"
basic.exec_sql(engine, sql)

data_dir = '../data/'
# sl_list = pd.read_csv( '../data/sl_hs_const.csv')
# sl_list = pd.read_csv( data_dir + 'sl_shengutong.csv')
# sl_list = pd.read_csv( '../data/sl_company.csv')
sl_list = pd.read_csv( data_dir + 'stocklist.csv')
sl_list.head(3), sl_list.shape
sl_size = len( sl_list )

# 从 tran_daily 中读出5条
# sql="SELECT * FROM sl_shengutong"
# sl_list = pd.read_sql_query(sql, conn)


In [86]:
sl_list.head(2), len(sl_list)

(     ts_code  name
 0  601318.SH  中国平安
 1  300474.SZ   景嘉微,
 2)

In [87]:
''' 数值运算 def  '''
def cal_upper(df):
    return ( df['high'] - max( df['open'], df['close'] ) ) / df['pre_close'] * 100
def cal_down(df):  
    return ( min( df['open'], df['close'] ) - df['low']  ) / df['pre_close'] * 100
def cal_range(df):  # 振幅%
    return ( df['high'] - df['low']  ) / df['pre_close'] * 100
def cal_st_high(df):  
    return df['open'] if df['open'] >= df['close'] else df['close']
def cal_st_low(df):
    return df['open'] if df['open'] < df['close'] else df['close']
def cal_st(df):  # 实体振幅
    return abs( df['open'] - df['close'] )
def cal_st_range(df):  # 实体振幅%
    return abs( df['open'] - df['close']  ) / df['pre_close'] * 100

''' 计算macd '''
# 输入df_name,算法参数。其中ewm函数算法快300-500倍
def cal_macd(dfIn, macd_way=2, ema_fast=12, ema_slow=26, dea_day=9):
    dfIn['ema_fast'], dfIn['ema_slow'] = 0.0, 0.0
    dfIn['diff'], dfIn['dea'], dfIn['bar'] = 0.0,0.0,0.0
    dfIn['ema_fast'] = dfIn['close'].ewm(span=ema_fast,     adjust=False).mean()
    dfIn['ema_slow'] = dfIn['close'].ewm(span=ema_slow,     adjust=False).mean()
    dfIn['diff'] = dfIn['ema_fast'] - dfIn['ema_slow']
    dfIn['dea'] = dfIn['diff'].ewm(span=9, adjust=False).mean()
    dfIn['bar'] = (dfIn['diff'] - dfIn['dea']) * 2
    return dfIn
    
def cal_ma(dfIn):
    dfIn['ma5'],dfIn['ma10'],dfIn['ma20'] = np.nan,np.nan,np.nan
    dfIn['ma30'],dfIn['MA48'] = np.nan,np.nan
    dfIn['ma5'] = dfIn['close'].rolling(5).mean()
    dfIn['ma10'] = dfIn['close'].rolling(10).mean()
    dfIn['ma20'] = dfIn['close'].rolling(20).mean()
    dfIn['ma30'] = dfIn['close'].rolling(30).mean()
    dfIn['MA48'] = dfIn['close'].rolling(48).mean()
    return dfIn

def add(*args):
    total = 0
    for val in args:
        total += val
    return total


In [88]:
def cal_pct_st():
    # 实体部分细化
    df['pct_upper'] = df.apply(cal_upper, axis=1)    # 上引线百分比
    df['pct_down']  = df.apply(cal_down,  axis=1)    # 下引线百分比
    df['pct_range'] = df.apply(cal_range, axis=1)    # 振幅百分比
    df['pct_st_range']  = df.apply(cal_st_range, axis=1)   # 实体部分百分比
    df['st_high']   = df.apply(cal_st_high,  axis=1)   # 实体高点价格
    df['st_low']    = df.apply(cal_st_low,   axis=1)   # 实体低点价格
    # df['pct_st']  = df.apply(cal_st, axis=1)   # 实体高度
    # 星线定义
    df['xing'] = df['pct_st_range']<=1
    return 'pct . '

def cal_macd():
    ''' way2 : ta-lib '''
    close = df.close.values  # 为 MACD 数据初始化close
    df['diff'], df['dea'], df['bar'] = ta.MACD(close, fastperiod=12, slowperiod=26, signalperiod=9)
    return 'macd . '

def cal_ma():
    df.insert( df.shape[1], 'ma3', ta.MA(df['close'], 3) )
    df.insert( df.shape[1], 'ma5', ta.MA(df['close'], 5) )
    df.insert( df.shape[1], 'ma10', ta.MA(df['close'], 10) )
    df.insert( df.shape[1], 'ma20', ta.MA(df['close'], 20) )
    df.insert( df.shape[1], 'ma30', ta.MA(df['close'], 30) )
    df.insert( df.shape[1], 'ma60', ta.MA(df['close'], 60) )
    df.insert( df.shape[1], 'ma120', ta.MA(df['close'], 120) )
    df.insert( df.shape[1], 'ma250', ta.MA(df['close'], 250) )
    return 'ma . '

def cal_vol_ma():
    df['vol_ma3']=df['volume'].rolling(3).mean()
    df['vol_ma5']=df['volume'].rolling(5).mean()
    df['vol_ma10']=df['volume'].rolling(10).mean()
    df['vol_ma20']=df['volume'].rolling(20).mean()
    df['vol_ma60']=df['volume'].rolling(60).mean()
    return 'vol_ma . '
 
# def cal_more():
    # # 股价变化均值
    # df.insert( df.shape[1], 'ma3_pct_change', ta.MA(df['pct_change'], 3) )
    # # N天之内最大股价
    # df['MAX3_pct_change']=df['pct_change'].rolling(3).max()
    # # n天之内最高收盘价
    # df['MAX90_high']=df['high'].rolling(90).max()

# def cal_boll():
#     df['bb-u'], df['bb-m'], df['bb-l'] = ta.BBANDS( close,timeperiod=20,nbdevup=2,nbdevdn=2,matype=0 )
#     df['upper'], df['middle'], df['lower'] = ta.BBANDS( close,timeperiod=20,nbdevup=2,nbdevdn=2,matype=0 )
    
# import
# df['MA15_pct_change'] = df['pct_change'].rolling(15).mean() 


In [97]:
# 均线系列函数， 

# 计算在60线以下的days
def jx_ud60_count(val):
    if val <= 0:
        ud60_count.count += 1
    else:
        # rolling_count.previous = val
        ud60_count.count = 0
    return ud60_count.count

# 均线的斜率, 输出度数
# def cal_upper(df):
#     return ( df['high'] - max( df['open'], df['close'] ) ) / df['pre_close'] * 100
def cal_jx_xl():
    # df['vol_ma3']=df['volume'].rolling(3).mean()
    # df['jx_xl_5'] = math.degrees( math.atan( (df['ma5'] - df['ma5'].rolling(3).mean() )/df['ma5']) )
    df['jx_xl_5']  = (df['ma5']  - df['ma5'].rolling(2).mean()) * 2/  df['ma5']
    df['jx_xl_10'] = (df['ma10'] - df['ma10'].rolling(2).mean()) * 2/ df['ma10']
    df['jx_xl_20'] = (df['ma20'] - df['ma20'].rolling(2).mean()) * 2/ df['ma20']
    df['jx_xl_30'] = (df['ma30'] - df['ma30'].rolling(2).mean()) * 2/ df['ma30']
    df['jx_xl_60'] = (df['ma60'] - df['ma60'].rolling(2).mean()) * 2/ df['ma60']
    df['jx_xl_120'] = (df['ma120'] - df['ma120'].rolling(2).mean()) * 2/ df['ma120']
    df['jx_xl_250'] = (df['ma250'] - df['ma250'].rolling(2).mean()) * 2/ df['ma250']
    return  'xl . '


   

In [98]:
%%time

print('计算各类指标 : ')
for i in range( len(sl_list)  ):
    
    # begin with ts_code
    print(sl_list.index[i]+1,' - ',str(sl_size), sl_list['ts_code'][i], end=" ")
    
    # 不计算8字头，北京股票
    if sl_list['ts_code'][i].startswith('8'):
        print( 'ignore 8xxx', end='\n' )
        continue
    # 不计算688字头，科创板
    if sl_list['ts_code'][i].startswith('688'):
        print( 'ignore 688x', end='\n' )
        continue
    
    # 本地MySQL读取
    sql = "select * from tran_daily where ts_code='"+sl_list['ts_code'][i]+"'"
    df = basic.read_data(conn, sql)
    df.set_index('trade_date', drop=True, append=False, inplace=True)
    df = df.sort_index(ascending=True)
    # 如果没有交易记录，则跳过loop
    if len(df) == 0:
        print( 'no data', end='' )
        continue  
    
    # 计算 percentage 和 实体部分
    print( cal_pct_st(), end='' )
    # 计算macd
    print( cal_macd(), end='' )
    # 计算ma
    print( cal_ma(), end='' )
    # 计算vol_ma
    print( cal_vol_ma(), end='' )

    # 计算junxian - 60线以下天数
    df['ud60_status'] = df['close'] - df['ma60']
    ud60_count.count = 0 #static variable
    df['jx_days_ud60'] = df['ud60_status'].apply(jx_ud60_count) #new column in dataframe
    del df['ud60_status']
    print( end='jx_ud60 . ' )

    # 计算cal_jx_xl - ma5,10,...250 均线斜率 
    print( cal_jx_xl(), end='' )

  
    # 空值填充为0
    df = df.fillna(value=0)

    """ 输出 to csv """
    # df.to_csv( data_dir+'cal/' + sl_list['ts_code'][i] +'.cal.csv', index=True )    
    ''' 输出到 mysql '''
    sql = "delete from day_zhibiao where ts_code='"+sl_list['ts_code'][i] +"'"
    basic.exec_sql(conn, sql)
    # df.to_sql('day_zhibiao', conn, index=True, if_exists='append', chunksize=2000)
    write_no = df.to_sql('day_zhibiao', conn, index=True, if_exists='append', chunksize=2000) # 20000个，每天成交几千只股票，足够了
    # index=True, 需要输出日期字段    
    # 首次运行，建立stock_cal表, 可以从df中建立表，表之前不存在，然后再改字段和index

    print( end='\n')

print('\n\n指标运算结束')

计算各类指标 : 
1  -  2 601318.SH pct . macd . ma . vol_ma . jx_ud60 . xl . 
2  -  2 300474.SZ pct . macd . ma . vol_ma . jx_ud60 . xl . 


指标运算结束
CPU times: total: 250 ms
Wall time: 1.24 s


In [91]:
df.head(2)

,ts_code,open,high,low,close,pre_close,change,pct_change,volume,value,...,ma60,ma120,ma250,vol_ma3,vol_ma5,vol_ma10,vol_ma20,vol_ma60,jx_xl_5,jx_days_ud60
trade_date,,,,,,,,,,,,,,,,,,,,,
20160720,300474.SZ,143.60,148.70,143.60,146.53,140.20,6.33,4.52,48090.4,706024.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
20160721,300474.SZ,144.65,146.91,141.18,141.30,146.53,-5.23,-3.57,33332.7,477900.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0


In [92]:
# df['jx_xl_5'] = (df['ma5'] - df['ma5'].rolling(3).mean()) / df['ma5']


In [93]:
df.head(10)[['close','ma5', 'jx_xl_5']]

,close,ma5,jx_xl_5
trade_date,,,
20160720,146.53,0.000,0.000000
20160721,141.30,0.000,0.000000
20160722,140.68,0.000,0.000000
20160725,136.41,0.000,0.000000
20160726,138.20,140.624,0.000000
20160727,124.90,136.298,-0.031739
20160728,122.44,132.526,-0.028462
20160729,120.88,128.566,-0.030801
20160801,114.82,124.248,-0.034753


In [94]:
df['jx_under60']=df['volume'].rolling(30).count( df['close']>df['ma60'] )


d:\temp\ipykernel_15540\3547900670.py:1: FutureWarning: min_periods=None will default to the size of window consistent with other methods in a future version. Specify min_periods=0 instead.
  df['jx_under60']=df['volume'].rolling(30).count( df['close']>df['ma60'] )


In [95]:
df1 = df[ df['jx_days_ud60']==0 ]
df1.head()
#筛选a列中值为a的行，形成新的df1
# df1 = df[ df['a'].isin(['a']) ]
#筛选列名为a、b、c的列，形成新的df1
# df1 = df[['a','b','c']]


,ts_code,open,high,low,close,pre_close,change,pct_change,volume,value,...,ma120,ma250,vol_ma3,vol_ma5,vol_ma10,vol_ma20,vol_ma60,jx_xl_5,jx_days_ud60,jx_under60
trade_date,,,,,,,,,,,,,,,,,,,,,
20160720,300474.SZ,143.60,148.70,143.60,146.53,140.20,6.33,4.52,48090.4,706024.0,...,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0,1.0
20160721,300474.SZ,144.65,146.91,141.18,141.30,146.53,-5.23,-3.57,33332.7,477900.0,...,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0,2.0
20160722,300474.SZ,141.50,144.90,139.88,140.68,141.30,-0.62,-0.44,24625.1,350293.0,...,0.0,0.0,35349.400000,0.0,0.0,0.0,0.0,0.0,0,3.0
20160725,300474.SZ,140.08,142.00,134.28,136.41,140.68,-4.27,-3.04,31953.0,438387.0,...,0.0,0.0,29970.266667,0.0,0.0,0.0,0.0,0.0,0,4.0
20160726,300474.SZ,136.60,138.95,135.00,138.20,136.41,1.79,1.31,20949.8,288675.0,...,0.0,0.0,25842.633333,31790.2,0.0,0.0,0.0,0.0,0,5.0


In [96]:
# 插入和删除行，很有用
# df.insert( df.shape[1], 'ma250', ta.MA(df['close'], 250) )
# df.drop( labels=['MA125'], axis=1, inplace=True)
# df.head(3)